# DS-SF-27 | Codealong 17 | Introduction to Databases | Answer Key

## RMDBS databases and `SQLite`

In [1]:
import os

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', 10)
pd.set_option('display.notebook_repr_html', True)
pd.set_option('display.max_columns', 10)

import sqlite3

> ### How to connect to a `SQLite` database

In [2]:
db = sqlite3.connect(os.path.join('..', 'datasets', 'zillow.db'))

> ### How to use `pandas` to run SQL queries and get the results as a `DataFrame`

In [3]:
pd.io.sql.read_sql(
'''
SELECT *
    FROM properties
    LIMIT 10
;
''', con = db)

,ID,address,latitude,longitude,is_a_studio,...,size,size_unit,lot_size,lot_size_unit,built_in_year
0,15063471,"55 Vandewater St APT 9, San Francisco, CA",37805103,-122412856,0,...,550.0,sqft,NaN,None,1980.0
1,15063505,"740 Francisco St, San Francisco, CA",37804420,-122417389,0,...,1430.0,sqft,2435.0,sqft,1948.0
2,15063609,"819 Francisco St, San Francisco, CA",37803728,-122419055,0,...,2040.0,sqft,3920.0,sqft,1976.0
3,15064044,"199 Chestnut St APT 5, San Francisco, CA",37804392,-122406590,0,...,1060.0,sqft,NaN,None,1930.0
4,15064257,"111 Chestnut St APT 403, San Francisco, CA",37804240,-122405509,0,...,1299.0,sqft,NaN,None,1993.0
5,15064295,"111 Chestnut St APT 702, San Francisco, CA",37804240,-122405509,0,...,1033.0,sqft,NaN,None,1993.0
6,15064391,"1821 Grant Ave APT 101, San Francisco, CA",37803748,-122408531,0,...,1048.0,sqft,NaN,None,1975.0
7,15064536,"2300 Leavenworth St, San Francisco, CA",37802408,-122417537,0,...,2115.0,sqft,1271.0,sqft,1913.0
8,15064640,"1047-1049 Lombard St, San Francisco, CA",37801889,-122418704,1,...,4102.0,sqft,3049.0,sqft,1948.0
9,15064669,"1055 Lombard St # C, San Francisco, CA",37801873,-122418834,0,...,1182.0,sqft,NaN,None,1986.0


### `SELECT`

In [4]:
pd.io.sql.read_sql(
'''
SELECT ID, sale_price, sale_price_unit
    FROM transactions
;
''', con = db)

,ID,sale_price,sale_price_unit
0,15063471,710000.00,$
1,15063505,2.15,$M
2,15063609,5.60,$M
3,15064044,1.50,$M
4,15064257,970000.00,$
...,...,...,...
995,2124214951,390000.00,$
996,2126960082,860000.00,$
997,2128308939,830000.00,$
998,2131957929,835000.00,$


### `WHERE`

In [5]:
pd.io.sql.read_sql(
'''
SELECT ID, sale_price
    FROM transactions
        WHERE sale_price_unit = '$'
;
''', con = db)

,ID,sale_price
0,15063471,710000.0
1,15064257,970000.0
2,15064295,940000.0
3,15064391,835000.0
4,15065032,800000.0
...,...,...
420,2124214951,390000.0
421,2126960082,860000.0
422,2128308939,830000.0
423,2131957929,835000.0


> #### Here's let's convert the unit of `sale_price` from \$M to \$

In [6]:
pd.io.sql.read_sql('''
SELECT ID, sale_price * 1000000 AS sale_price
    FROM transactions
        WHERE sale_price_unit = '$M'
;
''', con = db)

,ID,sale_price
0,15063505,2150000.0
1,15063609,5600000.0
2,15064044,1500000.0
3,15064536,2830000.0
4,15064640,4050000.0
...,...,...
570,2106229715,1240000.0
571,2108411930,1730000.0
572,2108546010,1400000.0
573,2121221385,1430000.0


### `UNION`

> #### Let's combine the previous two queries into one.  `sale_price` is now in \$ accross the dataset

In [7]:
pd.io.sql.read_sql('''
SELECT ID, sale_price
    FROM transactions
        WHERE sale_price_unit = '$'
UNION ALL
SELECT ID, sale_price * 1000000 AS sale_price
    FROM transactions
        WHERE sale_price_unit = '$M'
;
''', con = db)

,ID,sale_price
0,15063471,710000.0
1,15064257,970000.0
2,15064295,940000.0
3,15064391,835000.0
4,15065032,800000.0
...,...,...
995,2106229715,1240000.0
996,2108411930,1730000.0
997,2108546010,1400000.0
998,2121221385,1430000.0


### `WITH`

In [8]:
pd.io.sql.read_sql('''
WITH normalized_sale_prices AS
    (SELECT ID, sale_price
        FROM transactions
            WHERE sale_price_unit = '$'
    UNION ALL
    SELECT ID, sale_price * 1000000 AS sale_price
        FROM transactions
            WHERE sale_price_unit = '$M')

SELECT *
    FROM normalized_sale_prices
;
''', con = db)

,ID,sale_price
0,15063471,710000.0
1,15064257,970000.0
2,15064295,940000.0
3,15064391,835000.0
4,15065032,800000.0
...,...,...
995,2106229715,1240000.0
996,2108411930,1730000.0
997,2108546010,1400000.0
998,2121221385,1430000.0


### `JOIN`

In [9]:
pd.io.sql.read_sql('''
WITH normalized_sale_prices AS
    (SELECT ID, sale_price
        FROM transactions
            WHERE sale_price_unit = '$'
    UNION ALL
    SELECT ID, sale_price * 1000000 AS sale_price
        FROM transactions
            WHERE sale_price_unit = '$M')

SELECT properties.bed_count AS bed_count, normalized_sale_prices.sale_price AS sale_price
    FROM properties
        JOIN normalized_sale_prices ON normalized_sale_prices.ID = properties.ID
;
''', con = db)

,bed_count,sale_price
0,1.0,710000.0
1,2.0,970000.0
2,2.0,940000.0
3,1.0,835000.0
4,1.0,800000.0
...,...,...
995,3.0,1240000.0
996,2.0,1730000.0
997,2.0,1400000.0
998,NaN,1430000.0


### `WITH` (take 2)

In [10]:
pd.io.sql.read_sql('''
WITH normalized_sale_prices AS
    (SELECT ID, sale_price
        FROM transactions
            WHERE sale_price_unit = '$'
    UNION ALL
    SELECT ID, sale_price * 1000000 AS sale_price
        FROM transactions
            WHERE sale_price_unit = '$M'),

bed_count_normalized_sale_prices AS
    (SELECT properties.bed_count AS bed_count, normalized_sale_prices.sale_price AS sale_price
        FROM properties
            JOIN normalized_sale_prices
                ON normalized_sale_prices.ID = properties.ID)

SELECT * FROM bed_count_normalized_sale_prices
;
''', con = db)

,bed_count,sale_price
0,1.0,710000.0
1,2.0,970000.0
2,2.0,940000.0
3,1.0,835000.0
4,1.0,800000.0
...,...,...
995,3.0,1240000.0
996,2.0,1730000.0
997,2.0,1400000.0
998,NaN,1430000.0


### `GROUP BY` and `AVG`

> ### Let's now compute the average sale price of these properties based on their number of bedrooms

In [11]:
pd.io.sql.read_sql('''
WITH normalized_sale_prices AS
    (SELECT ID, sale_price
        FROM transactions
            WHERE sale_price_unit = '$'
    UNION ALL
    SELECT ID, sale_price * 1000000 AS sale_price
        FROM transactions
            WHERE sale_price_unit = '$M'),

bed_count_normalized_sale_prices AS
    (SELECT properties.bed_count AS bed_count, normalized_sale_prices.sale_price AS sale_price
        FROM properties
            JOIN normalized_sale_prices
                ON normalized_sale_prices.ID = properties.ID)

SELECT AVG(sale_price) AS mean_sale_price
    FROM bed_count_normalized_sale_prices
        GROUP BY bed_count
;
''', con = db)

,mean_sale_price
0,1.494858e+06
1,8.204245e+05
2,1.192398e+06
3,1.404323e+06
4,1.879330e+06
5,2.867720e+06
6,2.587222e+06
7,5.543167e+06
8,1.960000e+06
9,1.766667e+06


Note: `bed_count = 0` usually refers to studios but not always...

> ### Activity
> #### Compute the average sales price of studios

In [12]:
pd.io.sql.read_sql('''
WITH normalized_sale_prices AS
    (SELECT ID, sale_price
        FROM transactions
            WHERE sale_price_unit = '$'
    UNION ALL
    SELECT ID, sale_price * 1000000 AS sale_price
        FROM transactions
            WHERE sale_price_unit = '$M')

SELECT AVG(normalized_sale_prices.sale_price) AS mean_sale_price
    FROM normalized_sale_prices
        JOIN properties ON properties.ID = normalized_sale_prices.ID
            WHERE properties.is_a_studio = 1
''', con = db)

,mean_sale_price
0,1.464034e+06


> ### Activity
> #### Find the properties in the dataset that have different addresses but the same latitude/longitude.  Return the smallest set of pair of properties (`address_a`, `address_b`)

In [13]:
pd.io.sql.read_sql('''
SELECT a.address AS address_a, b.address as address_b,
    a.ID AS ID_a, b.ID as ID_b
        FROM properties AS a
            JOIN properties AS b
                ON (a.ID < b.ID) AND
                    (b.latitude = a.latitude) AND (b.longitude = a.longitude)
;
''', con = db)

,address_a,address_b,ID_a,ID_b
0,"111 Chestnut St APT 403, San Francisco, CA","111 Chestnut St APT 702, San Francisco, CA",15064257,15064295
1,"1070 Green St APT 201, San Francisco, CA","1070 Green St APT 1402, San Francisco, CA",15066001,15066036
2,"946 Stockton St APT 6B, San Francisco, CA","946 Stockton St APT 9E, San Francisco, CA",15067583,15067604
3,"1250 Jones St APT 503, San Francisco, CA","1250 Jones St APT 802, San Francisco, CA",15067890,15067896
4,"1177 California St APT 702, San Francisco, CA","1177 California St APT 501, San Francisco, CA",15068514,15068551
...,...,...,...,...
103,"2200 Market St APT 205, San Francisco, CA","2200 Market St APT 202, San Francisco, CA",124397844,124397860
104,"8 Octavia St UNIT 401, San Francisco, CA","8 Octavia St, San Francisco, CA",124850278,124862135
105,"1915 Diamond St, San Francisco, CA","1915 Diamond St, San Francisco, CA",125156575,2100850402
106,"2149 Lyon St APT 4, San Francisco, CA","2149 Lyon St APT 1, San Francisco, CA",2100643449,2100786271


## NoSQL databases and `CouchDB`

### Administration Web Interface

> `CouchDB`'s Administration Web Interface, called `Fauxton` is at http://localhost:5984/_utils

### A brief overview of `CouchDB`'s bare-bone API using the command-line utility `curl`

> #### Hello World!

In [14]:
!curl -X GET http://localhost:5984

{"couchdb":"Welcome","version":"2.0.0","vendor":{"name":"The Apache Software Foundation"}}


We now know that `CouchDB` is alive.  Let's keep going.

> #### Get the list of databases

In [15]:
!curl -X GET http://localhost:5984/_all_dbs

[]


There are none so far.  So let's go and create one.

> ### Create a new database called `zillow`

In [16]:
!curl -X PUT http://localhost:5984/zillow

{"ok":true}


> ### Check out the new database

In [17]:
!curl -X GET http://localhost:5984/zillow

{"db_name":"zillow","update_seq":"0-g1AAAAFTeJzLYWBg4MhgTmEQTM4vTc5ISXLIyU9OzMnILy7JAUoxJTIkyf___z8rkQGPoiQFIJlkT1idA0hdPGF1CSB19QTV5bEASYYGIAVUOp8YtQsgavcTo_YARO19YtQ-gKgFuTcLANRjby4","sizes":{"file":33952,"external":0,"active":0},"purge_seq":0,"other":{"data_size":0},"doc_del_count":0,"doc_count":0,"disk_size":33952,"disk_format_version":6,"data_size":0,"compact_running":false,"instance_start_time":"0"}


It's a new database so it is empty (`"doc_count":0`).  Let's now populate it.

> ### Bulk import

In [18]:
!curl -d @../datasets/zillow_pretty.json -H "Content-Type: application/json" -X POST http://localhost:5984/zillow/_bulk_docs

[{"ok":true,"id":"15063471","rev":"1-fc56be79dcb6417ee79ea8004659f480"},{"ok":true,"id":"15063505","rev":"1-db02dd0334a37e625110e1b9c8163482"},{"ok":true,"id":"15063609","rev":"1-c2e1f219ebe678e0a5da94bb7f993e8a"},{"ok":true,"id":"15064044","rev":"1-680741bddd49e5c7795e82c6aad4c35a"},{"ok":true,"id":"15064257","rev":"1-fb48c5841f54d328970bcebfe5cd9194"},{"ok":true,"id":"15064295","rev":"1-59f8f4fe67f161218660ccd6d9738f28"},{"ok":true,"id":"15064391","rev":"1-460378eaa953acd1b8e91f1b2b402f0a"},{"ok":true,"id":"15064536","rev":"1-6131e44751780be21d3ce21f274bb522"},{"ok":true,"id":"15064640","rev":"1-d389c46c88745cd4dc19ca912f800826"},{"ok":true,"id":"15064669","rev":"1-a1c64467b1c12008a0efcbe961d2a561"},{"ok":true,"id":"15065032","rev":"1-aa0c694d64ab1035cc293f50705f758b"},{"ok":true,"id":"15065140","rev":"1-136d6c3dda53b5d798468aa010b8d891"},{"ok":true,"id":"15065727","rev":"1-05128a6464bc39435d1f20668bc87f76"},{"ok":true,"id":"15065810","rev":"1-63ea22107982286626fad9959c326991"},{"ok"

Let's check it again...

In [19]:
!curl -X GET http://localhost:5984/zillow

{"db_name":"zillow","update_seq":"1000-g1AAAAFTeJzLYWBg4MhgTmEQTM4vTc5ISXLIyU9OzMnILy7JAUoxJTIkyf___z8rsQiPoiQFIJlkD1ZXjE-dA0hdPFjddHzqEkDq6sHqavGoy2MBkgwNQAqodH5WYhNBtQsgavcTo_YARO39rMRCgmofQNQC3VuVBQArrHMW","sizes":{"file":570880,"external":439367,"active":517672},"purge_seq":0,"other":{"data_size":439367},"doc_del_count":0,"doc_count":1000,"disk_size":570880,"disk_format_version":6,"data_size":517672,"compact_running":false,"instance_start_time":"0"}


We can see that we now have 1,000 documents in it (`"doc_count":1000`).  Let's have a look at a couple of documents.

> ### `_id`

In [20]:
!curl -X GET http://localhost:5984/zillow/15083161

{"_id":"15083161","_rev":"1-1379dbf0a11cccc3b47607ee4ea71f8a","long_id":"/homedetails/1825-Scott-St-San-Francisco-CA-94115/15083161_zpid/","address":"1825 Scott St, San Francisco, CA","map":{"latitude":37786681,"longitude":-122438711},"sale":{"date":"11/23/15","price":{"value":32.7,"unit":"$M"}},"is_a_studio":false,"bath_count":1.0,"size":{"value":1100.0,"unit":"sqft"},"lot_size":{"value":1694.0,"unit":"sqft"},"built_in":{"year":1900}}


That's our very expensive \$32M home!  15083161 refers to the Zillow identifier of our dataset but how did `CouchDB` know how to use it?  For the bulk import, we set the identifier of each document with `_id`.

In [21]:
!curl -X GET http://localhost:5984/zillow/15149005

{"_id":"15149005","_rev":"1-07a48bf1b96b7322faf07773eaab2769","long_id":"/homedetails/251-253-Missouri-St-San-Francisco-CA-94107/15149005_zpid/","address":"251-253 Missouri St, San Francisco, CA","map":{"latitude":37763202,"longitude":-122396270},"sale":{"date":"12/23/15","price":{"value":1.0,"unit":"$"}},"is_a_studio":false,"bed_count":7,"bath_count":6.0,"size":{"value":2904.0,"unit":"sqft"},"lot_size":{"value":2482.0,"unit":"sqft"},"built_in":{"year":1900}}


Our \$1 home...

## List of all documents within a database

In [22]:
!curl -X GET http://localhost:5984/zillow/_all_docs

{"total_rows":1000,"offset":0,"rows":[
{"id":"111710714","key":"111710714","value":{"rev":"1-f6187413a527d58fd6bc7b820b924e04"}},
{"id":"111711124","key":"111711124","value":{"rev":"1-157db6f4608474a1edb8a4068249e56a"}},
{"id":"111711143","key":"111711143","value":{"rev":"1-a71aa9588c16154ff90b4ad31c8600dd"}},
{"id":"111711144","key":"111711144","value":{"rev":"1-b1076fb2d1b8af763d3a28bbb27a0064"}},
{"id":"111926288","key":"111926288","value":{"rev":"1-cd4dc79a77394508be8bf565ebc35df5"}},
{"id":"111930969","key":"111930969","value":{"rev":"1-ec9c2fa259ba33ace7c5580dd9dc28ee"}},
{"id":"111931130","key":"111931130","value":{"rev":"1-14043db771d3740738d039acd5e15ec4"}},
{"id":"111932542","key":"111932542","value":{"rev":"1-a5d4e3c1878645aaacea410913e725cd"}},
{"id":"111934546","key":"111934546","value":{"rev":"1-1c9b1e71c2abea7714384da6c6562039"}},
{"id":"114314023","key":"114314023","value":{"rev":"1-f51fcf1661efb55597769aa79a5735d1"}},
{"id":"114317753","key":"114317753","value":{"rev":

> Now with the document contents

In [23]:
!curl -X GET http://localhost:5984/zillow/_all_docs?include_docs=true

{"total_rows":1000,"offset":0,"rows":[
{"id":"111710714","key":"111710714","value":{"rev":"1-f6187413a527d58fd6bc7b820b924e04"},"doc":{"_id":"111710714","_rev":"1-f6187413a527d58fd6bc7b820b924e04","long_id":"/homedetails/723-Taylor-St-501-San-Francisco-CA-94108/111710714_zpid/","address":"723 Taylor St # 501, San Francisco, CA","map":{"latitude":37789192,"longitude":-122412231},"sale":{"date":"12/2/15","price":{"value":798000.0,"unit":"$"}},"is_a_studio":false,"bed_count":1,"bath_count":1.0,"size":{"value":677.0,"unit":"sqft"},"built_in":{"year":2010}}},
{"id":"111711124","key":"111711124","value":{"rev":"1-157db6f4608474a1edb8a4068249e56a"},"doc":{"_id":"111711124","_rev":"1-157db6f4608474a1edb8a4068249e56a","long_id":"/homedetails/2125-Bryant-St-APT-5-San-Francisco-CA-94110/111711124_zpid/","address":"2125 Bryant St APT 5, San Francisco, CA","map":{"latitude":37760113,"longitude":-122409928},"sale":{"date":"11/17/15","price":{"value":1.38,"unit":"$M"}},"is_a_studio":false,"bed_count"

### Let's now using `CouchDB`'s `MapReduce` to compute the average sale price of these properties based on their number of bedrooms

> Using `Fauxton`, let's create a new view: http://localhost:5984/_utils/#/database/zillow/new_view

Copy/paste the following (JavaScript) code for the `Map function`.

```javascript
function(property) {
    if (property.bed_count) {
        bed_count = property.bed_count;
    } else if (property.is_a_studio == true) {
        bed_count = 0;
    } else {
        return;
    }
    if (property.sale.price.unit == '$') {
        emit(bed_count, [property.sale.price.value, 1]);
        return;
    }
    if (property.sale.price.unit == '$M') {
        emit(bed_count, [property.sale.price.value * Math.pow(10, 6), 1]);
        return;
    }
}
```

Copy/paste the following (JavaScript) code for the `Reduce (optional)` function (choose `CUSTOM`).

```javascript
function(keys, values) {
    weighted_price = 0
    weight = 0
        values.forEach(function (value) {
            weighted_price += value[0] * value[1];
            weight += value[1];
        });
     return [weighted_price / weight, weight];
}
```

Click on `Options` (upper right), check `Reduce` and `Run Query`.  Do these numbers match what we got from the `SQL` section?

### Delete a database

In [24]:
!curl -X DELETE http://localhost:5984/zillow

{"ok":true}
